In [156]:
%run Make_Question_simplified.ipynb
import numpy as np
from collections import defaultdict

import random
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import time
import csv

from collections import defaultdict

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import networkx as nx
import numpy as np
import dwave.inspector
import dimod

import matplotlib

from matplotlib import pyplot as plt

In [157]:
N = 2
#M = 5
I = 2
J = 2
K = 3
Data_upperbound = 20
Processing_upperbound = 20
Data_input = Data_Queue(I,J,Data_upperbound)
Data_input_type = Data_Queue_type(I,J,K)
VM_type = VM_Data_type_gen(I,K,N)

VM_speed = VM_Data_processing_speed(Processing_upperbound, VM_type, K)
VM_DP_time = VM_Data_processing_time(Data_input, Data_input_type , VM_speed)

modified_VM_DP_time =np.copy(VM_DP_time)
modified_VM_DP_time[np.isinf(modified_VM_DP_time)]=np.inf

T_max = little_greedy_algorithm(VM_DP_time,I,J,N)

In [158]:
print(VM_type)
print(Data_input_type)
print(VM_DP_time)
#print(np.min(VM_DP_time.reshape(I*J,N),1))
#print("T_max is ",T_max)
#print("let t = 1 unit of time")

[[2 3]
 [1 2]]
[[3 1]
 [1 3]]
[[[ 6. inf]
  [inf  2.]]

 [[inf  2.]
  [ 2. inf]]]


In [159]:
position_dictionary, Total = Make_dictionary(I,J,N,T_max)
QUBO_init = np.zeros((Total, Total))
P = {
    "Obj": 1,
    "C1":  20,
    "C2":  20,
    "C3":  20,
    "C4":  20,
    "C5":  50,
    "C6":  50,
    "C7":  60,
    "C8":  50,
    "C9":  50,
    "Cadd":  50,
    "Cadd1":  10,
    "Cadd2":  10,
}
QUBO_0 = mk_QUBO_0(I,J,N,T_max,QUBO_init,P)
QUBO_1 = mk_QUBO_1(I,J,N,T_max,QUBO_init,VM_type,Data_input_type,position_dictionary,P)
QUBO_2 = mk_QUBO_2(I,J,N,T_max,QUBO_init,VM_type,Data_input_type,position_dictionary,P)
QUBO_3 = mk_QUBO_3(I,J,N,T_max,QUBO_init,VM_type,Data_input_type,position_dictionary,P)
QUBO_4 = mk_QUBO_4(I,J,N,T_max,QUBO_init,VM_type,Data_input_type,position_dictionary,P)
QUBO_5 = mk_QUBO_5(I,J,N,T_max,QUBO_init,VM_type,Data_input_type,position_dictionary,modified_VM_DP_time,P)
QUBO_6 = mk_QUBO_6(I,J,N,T_max,QUBO_init,VM_type,Data_input_type,position_dictionary,modified_VM_DP_time,P)
QUBO_7 = mk_QUBO_7(I,J,N,T_max,QUBO_init,VM_type,Data_input_type,position_dictionary,modified_VM_DP_time,P)
#QUBO_8 = mk_QUBO_8(I,J,N,T_max,QUBO_init,VM_type,Data_input_type,position_dictionary,modified_VM_DP_time,P)
QUBO_add = mk_QUBO_add(I,J,N,T_max,QUBO_init,VM_type,Data_input_type,position_dictionary,modified_VM_DP_time,P)
QUBO_add_1 = mk_QUBO_add_1(I,J,N,T_max,QUBO_init,VM_type,Data_input_type,position_dictionary,modified_VM_DP_time,P)
QUBO_add_2 = mk_QUBO_add_2(I,J,N,T_max,QUBO_init,VM_type,Data_input_type,position_dictionary,modified_VM_DP_time,P)

QUBO = np.zeros_like(QUBO_init)
QUBO = QUBO_0 * P['Obj'] + QUBO_add + QUBO_1 + QUBO_2 + QUBO_3 + QUBO_4 + QUBO_5 + QUBO_6 + QUBO_7 + QUBO_add_1 + QUBO_add_2# + QUBO_8
#show_graph(QUBO) 
#print(len(position_dictionary))
QUBO,position_dictionary = mk_QUBO_8_alt(I,J,N,T_max,QUBO,VM_type,Data_input_type,position_dictionary,modified_VM_DP_time,P)
#print(len(position_dictionary),np.shape(QUBO))
QUBO,position_dictionary = mk_QUBO_9(I,J,N,T_max,QUBO,VM_type,Data_input_type,position_dictionary,modified_VM_DP_time,P)
#print(len(position_dictionary),np.shape(QUBO))


show_graph(QUBO_5) 

In [190]:
show_graph(QUBO_5) 

In [316]:
from dwave.system import LeapHybridSampler


width,height = np.shape(QUBO)

QUBO_dictionary = defaultdict(int)
for i in range(width):
    for j in range(height):
        QUBO_dictionary[(i,j)] = QUBO[i,j]
        
#sampler = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))
#sampler = EmbeddingComposite(DWaveSampler())
# Select a solver
sampler = LeapHybridSampler()

num_reads = 10000
#sampleset = sampler.sample_qubo(QUBO, chain_strength = 8, num_reads=num_reads)
#sampleset = sampler.sample_qubo(QUBO, num_reads=num_reads)
sampleset = sampler.sample_qubo(QUBO)


#sample = sampleset.first.sample
#energy = sampleset.first.energy
#print(sample,energy)
OP = sampleset.to_pandas_dataframe()
import pandas as pd
OP = OP.sort_values("energy")
OP_final = OP.to_numpy()

rank = 0

OP_opt = OP_final[rank,:].flatten()
sample = OP_opt.astype(int)



In [317]:
%run Make_Question_simplified.ipynb
#dir(sampleset)
print("Constraint_1: ",check_C1 (position_dictionary,sample,I,J,VM_type,Data_input_type))
print("Constraint_2: ",check_C2 (position_dictionary,sample,I,J,N,T_max))
print("Constraint_3: ",check_C3 (position_dictionary,sample,I,J,N,T_max))
print("Constraint_4: ",check_C4 (position_dictionary,sample,I,J,N,T_max))
print("Constraint_5: ",check_C5 (position_dictionary,sample,I,J,VM_type,Data_input_type,T_max,modified_VM_DP_time))
print("Constraint_6: ",check_C6 (position_dictionary,sample,I,J,N,T_max))
print("Constraint_7: ",check_C7 (position_dictionary,sample,I,J,N,T_max))
print("Constraint_8: ",check_C8_alt(position_dictionary,sample,I,J,VM_type,Data_input_type,T_max,modified_VM_DP_time))
print("Constraint_9: ",check_C9 (position_dictionary,sample,I,J,VM_type,Data_input_type,T_max))
print("Constraint_add: ",check_C_add(position_dictionary,sample,I,J,N,VM_type,Data_input_type,T_max))
print("Constraint_add_p: ",check_C_add_1(position_dictionary,sample,I,J,N,T_max))

print("Constraint_add_z: ",check_C_add_2(position_dictionary,sample,I,J,N,T_max))

[[1. 1.]
 [1. 1.]]
Constraint_1:  True
[[[1. 1.]
  [1. 1.]]

 [[1. 1.]
  [1. 1.]]]
Constraint_2:  True
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
Constraint_3:  True
[[[[1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]]

  [[1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]]]


 [[[1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]]

  [[1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1.]]]]
Constraint_4:  True
[1. 1. 1. 1. 1. 1. 1.]
Constraint_5:  True
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Constraint_6:  True
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Constraint_7:  True
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1.]
Constraint_8:  True
t is 0 t_prime is  1
p is 0 z is 0
t is 0 t_prime is  2
p is 0 z is 0
t is 0 t_prime is  3
p is 0 z is 0
t is 0 t_prime is  4
p is 0 z is 0
t is 1 t_prime is  2
p is 0 z is 0
t is 1 t_prime is  3
p 

In [291]:
OP

,0,1,2,3,4,5,6,7,8,9,...,150,151,152,153,154,155,156,157,energy,num_occurrences
0,1,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,-228.0,1


In [292]:
sampleset

SampleSet(rec.array([([1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -228., 1)],
          dtype=[('sample', 'i1', (158,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 10

In [293]:
opt_time = 0
for i in range(I):    
    for m in range(N):
        for t in range(T_max):
            print('p_%d%d%d%d value:'%(i,J-1,m,t), sample[position_dictionary['p_%d%d%d%d'%(i,J-1,m,t)]])
            
            opt_time += t * sample[position_dictionary['p_%d%d%d%d'%(i,J-1,m,t)]]
            

print("The opt_time is: ", opt_time)
print("the greedy time is: ", T_max)

p_0100 value: 0
p_0101 value: 0
p_0102 value: 0
p_0103 value: 0
p_0104 value: 0
p_0110 value: 0
p_0111 value: 1
p_0112 value: 0
p_0113 value: 0
p_0114 value: 0
p_1100 value: 0
p_1101 value: 1
p_1102 value: 0
p_1103 value: 0
p_1104 value: 0
p_1110 value: 0
p_1111 value: 0
p_1112 value: 0
p_1113 value: 0
p_1114 value: 0
The opt_time is:  2
the greedy time is:  5


In [301]:
for t in range(T_max):
    for m in range(N):
        work = np.zeros((I,J))
        for i in range(I):
            for j in range(J):
                #print('p_%d%d%d%d value:'%(i,J-1,m,t), sample[position_dictionary['p_%d%d%d%d'%(i,J-1,m,t)]])
            
                work[i,j] = sample[position_dictionary['y_%d%d%d%d'%(i,j,m,t)]] 
        people,index = np.where(work ==1)
        if len(people) > 0:
            print("when t = ", t ,": the machine", m ,"is working on", people[0],"th people's", index[0], "th workload.")
        else:
            print("when t = ", t ,": the machine", m ,"is hanging up")
print("The opt_time is: ", opt_time)
print("the greedy time is: ", T_max)

when t =  0 : the machine 0 is working on 1 th people's 1 th workload.
when t =  0 : the machine 1 is working on 0 th people's 1 th workload.
when t =  1 : the machine 0 is hanging up
when t =  1 : the machine 1 is hanging up
when t =  2 : the machine 0 is working on 0 th people's 0 th workload.
when t =  2 : the machine 1 is working on 1 th people's 0 th workload.
when t =  3 : the machine 0 is working on 0 th people's 0 th workload.
when t =  3 : the machine 1 is hanging up
when t =  4 : the machine 0 is hanging up
when t =  4 : the machine 1 is hanging up
The opt_time is:  2
the greedy time is:  5


In [308]:
for m in range(N):
    print("Machine",m)
    for t in range(T_max):
        for i in range(I):
            
            for j in range(J):        
                print("t = ",t,' z_%d%d value:'%(i,j), sample[position_dictionary['z_%d%d%d%d'%(i,j,m,t)]],' y_%d%d value:'%(i,j), sample[position_dictionary['y_%d%d%d%d'%(i,j,m,t)]],' p_%d%d value:'%(i,j), sample[position_dictionary['p_%d%d%d%d'%(i,j,m,t)]])


Machine 0
t =  0  z_00 value: 0  y_00 value: 0  p_00 value: 0
t =  0  z_01 value: 0  y_01 value: 0  p_01 value: 0
t =  0  z_10 value: 0  y_10 value: 0  p_10 value: 0
t =  0  z_11 value: 1  y_11 value: 1  p_11 value: 0
t =  1  z_00 value: 0  y_00 value: 0  p_00 value: 0
t =  1  z_01 value: 0  y_01 value: 0  p_01 value: 0
t =  1  z_10 value: 0  y_10 value: 0  p_10 value: 0
t =  1  z_11 value: 0  y_11 value: 0  p_11 value: 1
t =  2  z_00 value: 1  y_00 value: 1  p_00 value: 0
t =  2  z_01 value: 0  y_01 value: 0  p_01 value: 0
t =  2  z_10 value: 0  y_10 value: 0  p_10 value: 0
t =  2  z_11 value: 0  y_11 value: 0  p_11 value: 0
t =  3  z_00 value: 0  y_00 value: 1  p_00 value: 0
t =  3  z_01 value: 0  y_01 value: 0  p_01 value: 0
t =  3  z_10 value: 0  y_10 value: 0  p_10 value: 0
t =  3  z_11 value: 0  y_11 value: 0  p_11 value: 0
t =  4  z_00 value: 0  y_00 value: 0  p_00 value: 1
t =  4  z_01 value: 0  y_01 value: 0  p_01 value: 0
t =  4  z_10 value: 0  y_10 value: 0  p_10 value: 0
t 